In [1]:
import mapping_tools as mt
import numpy as np
import sys
import os
import gzip
import re
import pandas as pd
from timeit import default_timer as timer
from VEPcrossref import VEPfileCrossrefGenerator as cr

In [ ]:
# INTERFACES PART

In [2]:
# Extract the info corresponding to the prot ID (Interface parse)

#sys.argv[1]
#sys.argv[2]

protID = 'ENSP00000482258'
int_db_file = '/home/vruizser/PhD/2018-2019/Immunity_interfaces_analysis/raw_data/interfaces_mapped_to_v94.csv'

  
interfacesDB = open(int_db_file, "r")
cols = interfacesDB.readline().strip().split(" ")
#prot_interface = mt.ProtIntDB_parser(interfacesDB, protID)
prot_interface = mt.parser(interfacesDB,  protID, cols, " " )

In [10]:
subset_prot_interface = prot_interface[['pdb.id',
                      'ensembl.prot.id',
                      'temp.chain',
                      'int.chain',
                      'interaction',
                      'resid_sseq',
                      'mapped.real.pos',
                      'pdb.pos' ]]

# put it into right format
subset_prot_interface.columns = subset_prot_interface.columns.str.replace("\\.", "_")

for col in ("resid_sseq", "mapped_real_pos", "pdb_pos"):
    subset_prot_interface.loc[ : , col] = subset_prot_interface[col].str.replace("-", ',')
    subset_prot_interface.loc[ : , col] = subset_prot_interface[col].str.split(',')

subset_prot_interface = mt.explode(subset_prot_interface, ["resid_sseq", "mapped_real_pos", "pdb_pos"])

subset_prot_interface.rename(columns={'mapped_real_pos':'Protein_position'}, inplace=True)

subset_prot_interface['region_id'] = subset_prot_interface['pdb_id'] + '_' + subset_prot_interface['ensembl_prot_id'] + '_' + subset_prot_interface['temp_chain'] + '_' + subset_prot_interface['int_chain'] + '_' + subset_prot_interface['interaction']

In [ ]:
####################### VARIANTS PART #####################################################

In [32]:
# Translate protID to gene ID
#subset_prot_interface

In [4]:
biomartdb = open("/home/vruizser/PhD/2018-2019/git/PDBmapper/project/gene_transcript_protein_ens_ids.txt", "r")
ensemblIDs = mt.ensemblID_translator(biomartdb, protID)

In [5]:
# get the corresponding VEP file
crossref_file = open("/home/vruizser/PhD/2018-2019/git/PDBmapper/project/geneids_VEPfiles_crossref.txt", "r")
VEP_filename = mt.VEP_getter(crossref_file, ensemblIDs["geneID"])

In [6]:
VEP_dir= "/home/vruizser/PhD/2018-2019/PanCancer_data/vep_output/"
VEPfile = open(VEP_dir + VEP_filename, "r")
geneID = ensemblIDs['geneID']
cols = pd.read_csv(VEPfile, nrows = 0, skiprows = 42, sep = "\t").columns
VEP = mt.parser(VEPfile,  geneID, cols, "\t" )

In [9]:
setID = MapVariantToPDB(VEP, subset_prot_interface, 'region_id')

In [ ]:
# define main function to execute the previous defined functions together
def main():

 

    
    
    return


##########################
# execute main function  #
##########################
if __name__ == '__main__':
    main()

In [45]:
import vcfpy
import os
import re

In [130]:
input_vcf = '/home/vruizser/PhD/2018-2019/PanCancer_data/vep_output/vep_PanCan_chr_1_1-100000'
#input_vcf = "/home/vruizser/PhD/2019-2020/varMapDB/clinvar_20190828.vcf.gz"

#line = '#Uploaded_variation	Location	Allele	Gene	Feature	Feature_type	Consequence	cDNA_position	CDS_position	Protein_position	Amino_acids	Codons	Existing_variation	Extra'
#re.search(vep_header, line)
#print(  vep_header in line. split())
#result =  all(elem in vep_header  for elem in line.split())

#print(result)
def recognize_format(infile):
    try:
        vcf_reader = vcfpy.Reader.from_path(infile)
    
        if 'CSQ' not in vcf_reader.header.info_ids():
        #    vcf_reader.close()
            return print("ERROR: VCF {} is not VEP-annotated. Please annotate the VCF with VEP before running this tool.".format(infile))
        else : 
            return print('is VCF')
    except:
        print('Not in vcf format. Checking if VEP file')
        vep_header = set(['#Uploaded_variation',
                                'Location',
                                'Allele',
                                'Gene',
                                'Feature',
                                'Feature_type',
                                'Consequence',
                                'cDNA_position',
                                'CDS_position',
                                'Protein_position',
                                'Amino_acids',
                                'Codons',
                                'Existing_variation'])
            
        input_vcf= open(infile,'r')
        for line in input_vcf:
            jeje = set(line.split())
                
            res = vep_header.issubset(jeje)
            if res:
                return print("is vep format") 


        return print("input format not recognized")


recognize_format(input_vcf)

#except:
#    print('Incorrect input file format. The possible inputs are either VCF or VEP files')
#    exit(-1)

Not in vcf format. Checking if VEP file
is vep format


In [139]:
from scripts.detect_vcf_format import detect_format
import vcfpy
import subprocess
input_vcf = '/home/vruizser/PhD/2018-2019/PanCancer_data/vep_output/vep_PanCan_chr_1_1-100000'

print('Detecting inputs variants file format...')
input_format = detect_format(input_vcf)


Detecting inputs variants file format...
The input file is not in vcf format.               Checking if it is a VEP file...
is vep format


In [145]:
if input_format == "vcf":
    call('sh', './vcf_to_vep.sh', input_vcf )
    call('sh', 'split_vep_by_protid.sh', './data/converted_vcf.vep' )
elif input_format == "vep":
    cmd = "./home/vruizser/PhD/2018-2019/git/PDBmapper/scripts/split_vep_by_protid.sh " + input_vcf

    # no block, it start a sub process.
    p = subprocess.call(cmd)

    # and you can block util the cmd execute finish
    p

else: 
    print(input_format)

FileNotFoundError: [Errno 2] No such file or directory: './home/vruizser/PhD/2018-2019/git/PDBmapper/scripts/split_vep_by_protid.sh /home/vruizser/PhD/2018-2019/PanCancer_data/vep_output/vep_PanCan_chr_1_1-100000': './home/vruizser/PhD/2018-2019/git/PDBmapper/scripts/split_vep_by_protid.sh /home/vruizser/PhD/2018-2019/PanCancer_data/vep_output/vep_PanCan_chr_1_1-100000'